In [ ]:
import numpy as np
# from scipy import spatial
import torch 
import torch.nn as nn
import pandas as pd
from sklearn.metrics import mean_squared_error 
# torch.manual_seed(0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/python_data/社群網路與推薦系統/hw3/data/Movielens/user_movie.dat'

In [ ]:
files = ['Douban_Book', 'Movielens', 'Yelp']

In [ ]:
# with open(f'./data/Douban_Book/user_book.dat') as f:
#     data = [list(map(int, line.strip().split('\t'))) for line in f.readlines()]

# User-Item matrix 

CF 的缺點：

* 如果沒有用戶的歷史數據就沒辦法做任何推薦
* 以及無論 user-based 或 item-based 都需要消耗大量的運算資源
* 大部分用戶有評分紀錄的資料都只佔所有資料中的很小一部分，matrix 相當稀疏，很難找到相似的資料
* 會有馬太效應，越熱門的物品越容易被推薦，所以通常都會降低熱門物品的權重

user-based 考慮的是 user 和 user 之間的相似程度

    給定一個用戶 A
    計算用戶 A 跟其他所有用戶的相似度
    找出最相似的 m 個用戶
    再找出這些用戶有評分但是用戶 A 沒有評分的物品（也可以額外限制至少要幾個用戶有評分過）
    以「相似用戶的相似度」和「該用戶對該物品的評分」來加權算出用戶 A 對這些未評分物品的評分
    最後推薦給 A 評分最高的 n 個物品

預測 user_4 對 item_a 的評分 =
(user_4_user_1_sim x user_1_item_a_rating + user_4_user_3_sim x user_3_item_a_rating) / (user_4_user_1_sim + user_4_user_3_sim)

user-based 的特點：

* 適合 user 遠少於 item 的系統，相似度的計算量會較少
* item 的時效性強、更多樣的系統，例如新聞、社交網站，適合用 user-based CF
* 不容易給出推薦理由
* 驚喜度較高

## Filtering & Spliting
filtering 部分要改
* test split部分是對有評分的部分？
* Subtract mean的部分是針對user-item matrix不是sim_matrix

In [ ]:
class CF:
    def __init__(self, path= './data/Douban_Book/user_book.dat'):
        
        
        names = ['user_id', 'item_id', 'rating']
        df = pd.read_csv(path, sep='\t', names=names)
        
        '''filtering'''
        grouped_df = df.groupby(['user_id'])['item_id'].count()
        filtered_user_id = grouped_df[grouped_df>3].index
        filtered_df = df.set_index('user_id').loc[filtered_user_id].reset_index()
        
        '''user_id to index_id'''
        self.user_dict = dict()
        for i, user_id in enumerate(filtered_user_id):
            self.user_dict[user_id] = i 
        
        self.n_user= filtered_df['user_id'].unique().shape[0]
        self.n_item = filtered_df['item_id'].unique().shape[0]
        self.data= np.array(filtered_df)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            
    def k_fold_data_split(self):
        
        np.random.shuffle(self.data)
        train_data = self.data[:int(0.7*len(self.data))]
        val_data = self.data[int(0.7*len(self.data)):int(0.8*len(self.data))]
        test_data = self.data[int(0.8*len(self.data)):]
        return train_data, val_data, test_data
    
    def get_user_item_matrix(self):
        
        train_data, _, self.test_data= self.k_fold_data_split()
        matrix = torch.zeros((self.n_user, self.n_item), dtype= float)
        self.mask_matrix = torch.zeros((self.n_user, self.n_item), dtype= float).to(self.device)
        
        for info in train_data:
            matrix[self.user_dict[info[0]], info[1]-1] = info[2]
            self.mask_matrix[self.user_dict[info[0]], info[1]-1] = 1
        
        self.user_item_matrix = matrix 
        return matrix
    
    def get_similarity_matrix(self, method= 'cosine', kind= 'user', epsilon= 1e-9):
        
        user_item_matrix = self.get_user_item_matrix().to(self.device)
        # similarity_matrix = torch.zeros((self.n_user, self.n_user), dtype= float)
        if method == 'cosine':
            # epsilon -> small number for handling dived-by-zero errors
            if kind == 'user':
#                 sim = user_item_matrix.dot(user_item_matrix.T) + epsilon
                sim = torch.mm(user_item_matrix, user_item_matrix.t()) + epsilon
            elif kind == 'item':
                sim = user_item_matrix.T.dot(user_item_matrix) + epsilon
            norms = torch.sqrt(torch.diagonal(sim))
            sim_matrix = sim/norms/norms.t()
            # adjusted_sim_matrix = sim_matrix - torch.mean(sim_matrix, dim= 1) #減去平均
            return sim_matrix
        # elif method == 'Spearman':
          
            
    def k_nearest_neighbor(self, sim_matrix, user_id, item_id, k= 10):
      # sim_matrix= self.similarity_matrix()
      '''
      sim_matrix: [n_user, n_user], sim_matrix[self.user_dict[user_id]]：[n_user, 1]
      mask: n_user x n_item, mask[:, item_id]：[n_user, 1]
      '''
      top_k_neighbor_ids = torch.argsort(sim_matrix[self.user_dict[user_id]]*self.mask_matrix[:, item_id], dim= 0, descending=True)[1:k+1] #除了自己
      return top_k_neighbor_ids
    
    def predict(self):
      predicts = []
      ratings = []
      neighbor_dict = dict()
      sim_matrix = self.get_similarity_matrix()
      # unique_user_ids = list(set([info[0] for info in self.test_data]))
      # for user_id in unique_user_ids:
      #   neighbor_dict[user_id] = self.k_nearest_neighbor(sim_matrix= sim_matrix, user_id= user_id)

      for (user_id, item_id, rating) in self.test_data:
        # print(neighbor_dict[user_id]) #該user_id對應的k個最近鄰居
        '''該user_id對應有對item評分的最近k個鄰居'''
        k_neighbor_ids = self.k_nearest_neighbor(sim_matrix= sim_matrix, user_id= user_id, item_id= item_id)
        neighbor_ratings = self.user_item_matrix[k_neighbor_ids].t()[item_id] #k個鄰居在item對應的評分
        predict = torch.sum(neighbor_ratings).item()/len(neighbor_ratings)
        predicts.append(predict)
        ratings.append(rating)
      return mean_squared_error(ratings, predicts) 

## Main

In [ ]:
# CF_obj= CF(path= path)

In [ ]:
# CF_obj.predict()

------

# Matrix Factorization

n_latent factor k
  input:
  * user_factor_matrix p： [k, n_user]]
  * item_factor_mat q：[k, n_item]
  * r：mask_matrix
  * y：rating matrix

initialization：

  * learning rate
  * lambda (regularization) = beta
  * iteration 
  * init for p, q：~N(0, 0.01)
  * SGD for updating params 

Loss function：

for every r(i, j)= 1, minimize [user_mat matmul item_mat.t() - rating_matrix) + two regularization function corresponding to user, item factor matrix, respectively. 

[圖片](https://i.imgur.com/WB6Xllg.png)

Biases：

* subtract global mean u
* learn biases for user and items 
* rating = global mean + preference factor + item bias + user bias

In [ ]:
class user_item_matrix():
  def __init__(self, path):
    names = ['user_id', 'item_id', 'rating']
    df = pd.read_csv(path, sep='\t', names=names, usecols = [i for i in range(3)])
    
    '''filtering'''
    grouped_df = df.groupby(['user_id'])['item_id'].count()
    filtered_user_id = grouped_df[grouped_df>3].index
    filtered_df = df.set_index('user_id').loc[filtered_user_id].reset_index()
    
    '''user_id to index_id'''
    self.user_dict = dict()
    for i, user_id in enumerate(filtered_user_id):
        self.user_dict[user_id] = i 
    
    self.n_user= filtered_df['user_id'].unique().shape[0]
    self.n_item = filtered_df['item_id'].unique().shape[0]
    self.data= [[self.user_dict[info[0]], info[1], info[2]] for info in np.array(filtered_df)]
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

  def k_fold_data_split(self):
    
    np.random.shuffle(self.data)
    train_data = self.data[:int(0.7*len(self.data))]
    val_data = self.data[int(0.7*len(self.data)):int(0.8*len(self.data))]
    test_data = self.data[int(0.8*len(self.data)):]
    return train_data, val_data, test_data

  def get_user_item_matrix(self):
    
    train_data, val_data, test_data= self.k_fold_data_split()
    matrix = torch.zeros((self.n_user, self.n_item), dtype= float, device= self.device)
    test_matrix = torch.zeros((self.n_user, self.n_item), dtype= float, device= self.device)
    # mask_matrix = torch.zeros((self.n_user, self.n_item), dtype= float).to(self.device)
    for info in train_data:
      matrix[info[0], info[1]-1] = info[2]
    for info in test_data:
      test_matrix[info[0], info[1]-1] = info[2]
    return matrix, test_matrix

In [ ]:
class MF(nn.Module):
  def __init__(self, n_user, n_item, k, beta, device):
    super(MF, self).__init__()
    self.device = device
    self.n_user= n_user
    self.n_item= n_item
    self.beta = beta
    """parameters | .to(device) creates new tensor so that the params cant be fetched from model.parameters()"""
    self.b = 
    self.p = torch.nn.Parameter(torch.randn(size= (self.n_user, k), dtype= float, device= device))
    self.q = torch.nn.Parameter(torch.randn(size= (self.n_item, k), dtype= float, device= device))
    self.b_u = torch.nn.Parameter(torch.zeros(size=(self.n_user, 1), dtype= float, device= device))
    self.b_i = torch.nn.Parameter(torch.zeros(size=(self.n_item, 1), dtype= float, device= device)) 
    # self.params = nn.ParameterList([self.p, self.q, self.b_u, self.b_i])
  
  def forward(self, b):
    predict = b + self.b_u.repeat(1, self.n_item) + self.b_i.t().repeat(self.n_user, 1) + torch.matmul(self.p, self.q.t())
    return predict

  def loss_function(self, y, y_mask, mask_boolean, b, predict):

    # mask_boolean = (y_mask > 0)

    sse = torch.sum((torch.masked_select(y, mask_boolean) - torch.masked_select(predict, mask_boolean))**2)
    p_norm = torch.norm(self.p, dim= 1).unsqueeze(dim= 1) #[n_user, 1]
    q_norm = torch.norm(self.q, dim= 1).unsqueeze(dim= 1) #[n_item, 1]
    sum_b_u = torch.sum((torch.masked_select(self.b_u.repeat(1, self.n_item), mask_boolean))**2)
    sum_b_i = torch.sum((torch.masked_select(self.b_i.t().repeat(self.n_user, 1), mask_boolean))**2)
    sum_b = (b**2) * torch.sum(y_mask)
    sum_p_norm = torch.sum((torch.masked_select(p_norm.repeat(1, self.n_item), mask_boolean))**2)
    sum_q_norm = torch.sum((torch.masked_select(q_norm.t().repeat(self.n_user, 1), mask_boolean))**2)

    return 0.5*sse + self.beta * 0.5 * (sum_b_u + sum_b_i + sum_b + sum_p_norm + sum_q_norm)
  
  def RMSE(self, test_mat, test_mask, predict): 
    # test_mat: [n_user, n_item] float
    # test_mask: [n_user, n_item] boolean 
    # predict: [n_user, n_item] float

    return round(mean_squared_error(torch.masked_select(test_mat, test_mask).detach().cpu(), torch.masked_select(predict, test_mask).detach().cpu(), squared= False), 2)
    


## Main

* 每次產生user_item_matrix皆會進行隨機split

In [ ]:
lr= 5e-2
iteration = 100
k= 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
y, test_y= user_item_matrix(path= path).get_user_item_matrix()
y_mask = torch.where(y==0, y, 1.)
y_mask_boolean = (y > 0)
test_mask = (test_y > 0)
b = torch.sum(y.flatten())/torch.sum(y_mask)
n_user, n_item = y.shape
print(f'len of train_data|n_rating for training: {torch.sum(y_mask)}')

len of train_data|n_rating for training: 552228.0


In [ ]:
model = MF(n_user= n_user, n_item= n_item, k=k, beta=2, device= device)
# predict = model(user_item_matrix= matrix)

In [ ]:
# list(model.parameters())

## Training

In [ ]:
torch.cuda.empty_cache()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
model.train()
for i in range(iteration):
  # print(f'{i}th iteration')
  optimizer.zero_grad()
  output = model(b= b)
  # loss = model.loss_function(y= matrix, y_mask= y_mask, mask_boolean= y_mask_boolean, b= b, predict= output)
  loss = nn.MSELoss()(torch.masked_select(output, y_mask_boolean), torch.masked_select(y, y_mask_boolean))
  loss.backward()
  optimizer.step()
  torch.cuda.empty_cache()
  if (i+1)%10 == 0:
    print(f'loss: {loss.item():.3f}')
    print(model.RMSE(test_mat= test_y, test_mask= test_mask, predict= output))
  

loss: 4.609
2.15
loss: 4.604
2.15
loss: 4.599
2.15
loss: 4.594
2.14
loss: 4.589
2.14
loss: 4.583
2.14
loss: 4.578
2.14
loss: 4.573
2.14
loss: 4.568
2.14
loss: 4.563
2.14


In [ ]:
output

tensor([[3.6009, 4.7401, 3.7409,  ..., 4.4466, 5.0856, 7.3598],
        [4.0829, 5.3592, 4.4615,  ..., 3.6873, 6.0461, 5.9408],
        [4.7636, 3.0251, 4.8330,  ..., 6.7923, 1.7126, 3.4029],
        ...,
        [5.1858, 4.9946, 5.6265,  ..., 4.2929, 6.3073, 3.7461],
        [2.7848, 4.6068, 3.1156,  ..., 5.2084, 1.5004, 7.1326],
        [4.3528, 2.6998, 3.8131,  ..., 4.0002, 3.7332, 2.2705]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
y

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [4., 0., 3.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float64)